# Construção do dataset

## Padronização dos arquivos de dados climáticos do INMET

Colunas com diferentes formatações de dados de acordo com o ano, nomes de colunas diferentes de um arquivo para o outro, etc

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('dados_inmet.csv', sep=';', encoding='cp1252')
df.dtypes

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'Data': 'data',
                        'Hora (UTC)': 'hora',
                        'Temp. Max. (C)': 'tempmax',
                        'Temp. Min. (C)': 'tempmin',
                        'Umi. Max. (%)': 'umidmax',
                        'Umi. Min. (%)': 'umidmin',
                        'Vel. Vento (m/s)': 'vento',
                        'Chuva (mm)': 'chuva'})

# Remoção de colunas que não serão utilizadas
df = df.drop(columns=['Temp. Ins. (C)', 'Umi. Ins. (%)',
                      'Pto Orvalho Ins. (C)', 'Pto Orvalho Max. (C)',
                      'Pto Orvalho Min. (C)', 'Pressao Ins. (hPa)', 'Pressao Max. (hPa)',
                      'Pressao Min. (hPa)', 'Dir. Vento (m/s)', 'Raj. Vento (m/s)',
                      'Radiacao (KJ/mÂ²)'])
df.columns

In [ ]:
# Convertendo a coluna 'Data' para o tipo datetime
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

# Convertendo tipo 'object' para 'float'
for i in df.columns:
    if(df[i].dtype == object):
        df[i] = df[i].str.replace(',','.').astype(float)

df.set_index('data', inplace=True)

### Conversão de dados diários para semanais

In [ ]:
df_semanal = df.resample('W').agg({'tempmin': 'min',
                                    'umidmin': 'min',
                                    'tempmax': 'max',
                                    'umidmax': 'max',
                                    'vento': 'max',
                                    'chuva': 'sum'})
df_semanal.reset_index(inplace=True)
df_semanal

## Dados do InfoDengue
Concatenação entre dados sobre número de casos e incidência com dados climáticos

In [ ]:
infodengue = pd.read_csv('downloadedInfoDengueData.CSV')
infodengue = infodengue.rename(columns={'data_iniSE_sm':'data'})
infodengue

In [ ]:
infodengue['data'] = pd.to_datetime(infodengue['data'], format='%Y-%m-%d')
df_final = pd.merge(infodengue, df_semanal, on='data', how='inner')
df_final

## Gráficos

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(figsize=(15, 6))

inicio_intervalo = 2020
fim_intervalo = 2022

#df_ano = df_final[df_final['data'].dt.year < ano]
df_ano = df_final[(df_final['data'].dt.year >= inicio_intervalo) & (df_final['data'].dt.year <= fim_intervalo)]


# Plot the data from the first DataFrame
#ax.plot(df_ano['data'], df_ano['tempmin'], label='tempmin')
ax.plot(df_ano['data'], df_ano['tempmax'], label='tempmax')
ax.plot(df_ano['data'], df_ano['chuva']*0.2, label='chuva')
ax.plot(df_ano['data'], df_ano['casos_sm']*0.2, label='chuva')

# Add labels and legend
ax.set_xlabel('Data')
ax.set_ylabel('Temp')

# Show the plot
plt.legend()
plt.show()

#Exportar para drive

In [ ]:
# Mover o arquivo CSV para o Google Drive
df_final.to_csv('entrada_modelos.CSV', index=False)